In [2]:
import numpy as np
import open3d as o3d

# Robot Definition
The robot can be seen in the following image:

TODO: add image

It has 5 joints in a 2DoF shoulder, 1DoF elbow, 2DoF wrist configuration (the 2 wrist joints are both perpendicular to the elbow, while the elbow is parallel with the second shoulder joint).

In DH notation, the robot is defined as follows:

In [ ]:
# Define